In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("output_mar.csv")
props = pd.read_csv("shape_properties_mar.csv")

In [ ]:
aodSums = None
AODs = [column for column in df.columns if "aod" in column and column != "aod550"]

for aod in AODs:
    if aodSums is None:
        aodSums = df[aod].copy()
    else:
        aodSums += df[aod]
        
for aod in AODs:
    df[aod] /= aodSums
    
humid = df["t2m"].copy()
dew = df["d2m"]

humid = np.exp(-2.501e6*(humid - dew)/(461.5*humid*dew))

df["hum"] = humid

df["msl"] /= 101325.0 # Pa -> atm (Standard atmosphere at sea level)

df["t2m"] /= 273.15 # 0 Celsius at 1.0
df["d2m"] /= 273.15 # 0 Celsius at 1.0

# kg/m^3 -> µg/m^3
df["pm1"] *= 10**9
df["pm2p5"] *= 10**9
df["pm10"] *= 10**9

# To nearest unit
tcs = [column for column in df.columns if column[:2] == "tc"]
for tc in tcs:
    maxLog = np.log10(df[tc].max())//3
    
    maxLog *= 3
    
    df[tc] *= 10**-maxLog

df["time"] = pd.to_datetime(df["time"])

df = df.sort_values(["shapeID","time"])

df.describe()

In [ ]:
df.hist(figsize=(36,36))

In [ ]:
df_std = df.copy()
pms = ["pm1","pm2p5","pm10"]

for tc in tcs:
    df_std[tc] = np.log10(df_std[tc] + max(1e-15,min(df_std[tc])))

for pm in pms:
    df_std[pm] = np.log10(df_std[pm] + max(1e-15,min(df_std[pm])))

df_std["aod550"] = np.log(df_std["aod550"] + max(1e-15,min(df_std["aod550"])))

In [ ]:
df_std.hist(figsize=(36,36))

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def plot_variance(x, max=20):
    explained_var = []
    for i in range(max):
        ratio = PCA(n_components=i).fit(x).explained_variance
        explained_var.push(ratio)
    plt.plot(explained_var, range(max))
    plt.show()

In [ ]:
ind_vars = ["pm2p5","aod550","t2m","d2m","msl"]

ind_vars.extend(tcs)

plot_variance(df[ind_vars].values)

In [ ]:
def plot_variance_single(x, max=10):
        explained_var = PCA(n_components=max).fit(x).explained_variance_ratio_
        plt.plot(range(1,max+1),np.cumsum(explained_var))
        plt.show()